<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0515iwashita_yoshihara_BERT_mlm_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 岩下，吉原勉強会資料
- date: 2022_0515
- filename: `2022_0515iwashita_yoshihara_BERT_mlm_demo.ipynb`
- memo: BERT を用いたマスク化言語モデルによる穴埋め問題の回答や選択肢作成に向けて

In [ ]:
# 必要なライブラリのインストール
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install transformers   #transformers==4.5.0
    !pip install fugashi        #fugashi==1.1.0
    !pip install ipadic        #ipadic==1.0.0

In [ ]:
# 訓練済言語モデルの読み込み
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
#model_name = "sonoisa/sentence-bert-base-ja-mean-tokens-v2"  # <- v2です。
#参照 https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
# とはいえ，旧バージョンの方が，納得できる結果を出す場合があるので，要検討である。

tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")

In [ ]:
# トークン化のテスト実施
inp_texts = ['今日は[MASK]へ行く。', 
             'ジュースをお願いします', '[MASK]をお願いします',   # ミンニチテキストより
             '宇宙ステーションはどこにあるんですか。',           # ミンニチテキストより
             '宇宙は重力がありませんから、歩くことができないんです。', # ミンニチテキストより
             '宇宙は[MASK]力がありませんから、歩くことができないんです。', # ミンニチテキストより
             '今日は[MASK]へ行く。',
]

inp_text = '今日は[MASK]へ行く。'
inp_tokens = tokenizer.tokenize(inp_text)
print(inp_tokens)

for inp_text in inp_texts:
    inp_tokens = tokenizer.tokenize(inp_text)
    print(inp_tokens)
    print('---')


In [ ]:
# `encode()` 関数に `inp_text` 文章を渡して，言語モデルによって符号化された系列 `input_ids` を得る
input_ids = tokenizer.encode(inp_text, return_tensors='pt')

# 系列長を揃える必要がないので，単に iput_ids のみを入力する。
# 複数のテキストを処理させるときには max_length が必要となる
with torch.no_grad():
    output = bert_mlm(input_ids=input_ids)
    scores = output.logits

In [ ]:
# ID 列で '[MASK]' (ID は 4) の位置を調べて mask_position に保存
mask_position = input_ids[0].tolist().index(4) 
# index(4) という 魔法の数の招待は，直下行のと，次のセルを参照
#mask_position = input_ids[0].tolist().index(tokenizer.convert_tokens_to_ids('[MASK]'))

# 得点が最も良いトークンの ID を取り出してトークンに変換し，`id_best` に格納
id_best = scores[0, mask_position].argmax(-1).item()   # `argmax()` 関数の最終項が最大値なので，その値を `is_best` に格納
token_best = tokenizer.convert_ids_to_tokens(id_best)  # 直上行で計算された ID 番号 `is_best` (整数値) を tokenizer を使ってトークンに変換
token_best = token_best.replace('##', '')              # BPE の断片を変換する

# [MASK]を上で求めたトークンで置き換える。
inp_text = inp_text.replace('[MASK]', token_best)
print(inp_text)

In [ ]:
# ちなみに BERT には次のような特殊トークンがあります
print(tokenizer.special_tokens_map)
print(f"すなわち [MASK] の ID 番号は {tokenizer.convert_tokens_to_ids('[MASK]')} です")

In [ ]:
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk=4):
    """
    文章中の最初の [MASK] を得点上位のトークンに置き換える。
    上位何位まで使うかは num_topk で指定します。
    出力は穴埋めされた文章のリストと，置き換えられたトークンのスコアのリスト。
    """
    # 文章を符号化し BERT で分類得点を算出
    input_ids = tokenizer.encode(text, return_tensors='pt')
    #input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # 得点上位のトークンと対応する得点を求める。
    mask_position = input_ids[0].tolist().index(4)  # [MASK] トークンの ID は 4
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の[MASK]を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))

    return text_topk, scores_topk

inp_texts = ['今日は[MASK]へ行く。', '[MASK]をお願いします', '宇宙[MASK]はどこにあるんですか。', '宇宙は[MASK]がありませんから、歩くことができないんです。']
for inp_text in inp_texts:
    text_topk, _ = predict_mask_topk(inp_text, tokenizer, bert_mlm, num_topk=5)
    print(*text_topk, sep='\n')
    print('---')

In [ ]:
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK] トークンを含む文章を入力として，貪欲法で穴埋めを行った文章を出力
    """
    # 前から順に [MASK] を一つづつ得点の最も高いトークンに置き換える
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0]
    return text

inp_texts = ['[MASK]をお願いし[MASK]', '宇宙[MASK]は[MASK]にあるんですか。', '宇宙は[MASK]がありませんから、歩くことができないんです。']

for inp_text in inp_texts:
    print(greedy_prediction(inp_text, tokenizer, bert_mlm))

In [ ]:
inp_texts = ['今日は[MASK][MASK][MASK][MASK][MASK]', '宇宙[MASK]は[MASK][MASK]あるんですか。', '宇宙は[MASK]がありませんから、[MASK][MASK]ができないん[MASK]。']

for inp_text in inp_texts:
    print(greedy_prediction(inp_text, tokenizer, bert_mlm))

In [ ]:
def beam_search(text, tokenizer, bert_mlm, num_topk=10):
    """ビームサーチで文章の穴埋めの候補項目を探索して表示"""
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for _ in range(num_mask):
        # 現在得られている、それぞれの文章に対して
        # 最初の [MASK] をスコアが上位のトークンで空欄補充する。
        # 問題作成の際に，上位項目を選べば，選択肢の半自動作成になるのではないだろうか。
        text_candidates = []       # それぞれの文章を穴埋めした結果を追加する。
        score_candidates = []      # 穴埋めに使ったトークンのスコアを追加する。
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append( score + scores_topk_inner )

        # 穴埋めにより生成された文章の中から合計スコアの高い項目を選択
        score_candidates = np.hstack(score_candidates)
        idx_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [ text_candidates[idx] for idx in idx_list ]
        scores_topk = score_candidates[idx_list]

    return text_topk

inp_texts = ["今日は[MASK][MASK]へ行く。", '宇宙は[MASK]がありませんから、歩くことができないんです。']
for inp_text in inp_texts:
    text_topk = beam_search(inp_text, tokenizer, bert_mlm, num_topk=4)
    print(*text_topk, sep='\n')
    print('---')

In [ ]:
inp_texts = ['今日は[MASK][MASK][MASK][MASK][MASK]', '宇宙は[MASK]がありませんから、[MASK]ことができないんです。']

for inp_text in inp_texts:
    text_topk = beam_search(inp_text, tokenizer, bert_mlm, 10)
    print(*text_topk, sep='\n')
    print('---')

ここから下は著作権の問題があリます。
ですので，公表する場合には配慮が必要です。

In [ ]:
# GitHub からデータを持ってくる
!git clone https://github.com/ShinAsakawa/ccap.git
with open('ccap/2022_0515minnchi_sents.txt', 'r', encoding='utf-8') as f:
    _lines = f.readlines()
minnichi_sents = [line.strip() for line in _lines]
print(minnichi_sents[:3])


In [ ]:
def masked_minnichi(sentence_num=0, bert_mlm=bert_mlm, num_topk=4):
    
    # 第一引数がマイナスだったら，乱数を用いて文を一つ選ぶ
    if sentence_num > len(minnichi_sents) or (sentence_num < 0):
        sentence_num = np.random.choice(len(minnichi_sents))
    inp_text = minnichi_sents[sentence_num]
    inp_tokens = tokenizer.tokenize(inp_text)
    print(f'オリジナル:{inp_tokens}')

    target_pos = np.random.choice(len(inp_tokens))  # 乱数を用いて，ランダムな位置を選ぶ
    target_token_id = inp_tokens[target_pos]        # 元の単語 ID を保存
    inp_tokens[target_pos] = '[MASK]'               # 選んだ位置の単語を [MASK] で置き換え
    print(f'単語置き換え後の文:{inp_tokens}')        # 置き換えた文を表示
    input_ids = tokenizer.encode(inp_tokens, return_tensors='pt')
    print(f'単語置き換え後の input_ids:{input_ids}')

    with torch.no_grad():
        output = bert_mlm(input_ids=input_ids)
    scores = output.logits

    # 得点上位のトークンと対応する得点を求める。
    topk = scores[0, target_pos].topk(num_topk)
    ids_topk = topk.indices # トークン ID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk) # トークン
    scores_topk = topk.values.cpu().numpy() # スコア

    # 文章中の [MASK] を上で求めたトークンで置き換える。
    text_topk = [] # 穴埋めされたテキストを追加する。
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(inp_text.replace('[MASK]', token, 1))

    return text_topk, target_pos, target_token_id, output, scores, 

masked_minnichi(sentence_num=-1)[0]
